In [1]:
cd /content/drive/My Drive/Colab Notebooks/MaskRCNN_brain_mri

/content/drive/My Drive/Colab Notebooks/MaskRCNN_brain_mri


In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-5t46bkg8
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-5t46bkg8
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266454 sha256=e8436a0d7d4ee65a97fbbab517cc7d231c6de3595494b1d02cc4d31b4ba9a78e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6z00rgvp/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.1
    Uninstalling pycocotools-2.0.1:
      Successfully uninstalled pycocotools-2.0.1


In [ ]:
import os.path as osp
import csv
from dataloader import make_datapath_list, BrainDataset, get_transform

# rootpath = osp.abspath("")
# makepath = make_datapath_list(rootpath)
# train_img_list, train_anno_list = makepath.get_list("train")
# val_img_list, val_anno_list = makepath.get_list("val")

def make_path_csv(csv_path):
  csv_file = open(csv_path, "r", encoding="ms932", errors="", newline="" )
  rows = csv.reader(csv_file ,delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
  img_paths = []
  anno_paths = []
  for row in rows:
    img_paths.append(row[1])
    anno_paths.append(row[2])
  return img_paths[1:], anno_paths[1:]

train_img_list, train_anno_list = make_path_csv("train_path.csv")
val_img_list, val_anno_list = make_path_csv("validation_path.csv")
print("train dataset:", len(train_img_list),"validation dataset",len(val_img_list))

train dataset: 945 validation dataset 221


In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
import torch
import utils

#今回は他のモデルも前処理を加えていないのでtrainをFalseにして学習させます。
dataset = BrainDataset(train_img_list, train_anno_list, get_transform(train=False))
dataset_test = BrainDataset(val_img_list, val_anno_list, get_transform(train=False))
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=4, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2

model = get_instance_segmentation_model(num_classes)
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


In [ ]:
from engine import train_one_epoch, evaluate
# let's train it for 10 epochs
num_epochs = 20

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

    if (epoch+1) % 5 == 0:
      torch.save(model.state_dict(), 'weights/maskrcnn_resnet50_fpn_' + str(epoch+1) + '.pth') 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.6/dist-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


Epoch: [0]  [  0/237]  eta: 0:31:56  lr: 0.000026  loss: 5.0819 (5.0819)  loss_classifier: 0.6536 (0.6536)  loss_box_reg: 0.0844 (0.0844)  loss_mask: 4.2879 (4.2879)  loss_objectness: 0.0515 (0.0515)  loss_rpn_box_reg: 0.0046 (0.0046)  time: 8.0860  data: 6.2609  max mem: 3021
Epoch: [0]  [ 10/237]  eta: 0:07:54  lr: 0.000238  loss: 3.1216 (3.0712)  loss_classifier: 0.5527 (0.4880)  loss_box_reg: 0.0807 (0.0811)  loss_mask: 2.4078 (2.4349)  loss_objectness: 0.0660 (0.0608)  loss_rpn_box_reg: 0.0051 (0.0064)  time: 2.0889  data: 1.2353  max mem: 3303
Epoch: [0]  [ 20/237]  eta: 0:06:29  lr: 0.000449  loss: 1.3895 (2.1409)  loss_classifier: 0.1385 (0.3033)  loss_box_reg: 0.0711 (0.0783)  loss_mask: 1.1582 (1.6996)  loss_objectness: 0.0493 (0.0539)  loss_rpn_box_reg: 0.0051 (0.0057)  time: 1.4784  data: 0.7186  max mem: 3303
Epoch: [0]  [ 30/237]  eta: 0:05:12  lr: 0.000661  loss: 0.8181 (1.6869)  loss_classifier: 0.1020 (0.2408)  loss_box_reg: 0.0793 (0.0855)  loss_mask: 0.6000 (1.3080) 